In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
pd.set_option("display.max_rows", 2000, "display.max_columns", None)



def get_submission(pred, Name):
    res = pd.DataFrame(pred, index = test[test.columns[0]], columns=["target"]) 
    res.to_csv(str(Name)+ ".csv")

In [2]:
train = pd.read_csv(r"Train.csv")
test = pd.read_csv(r"Test.csv")

In [12]:
# train.isna().sum()
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,0

In [4]:
train.corr()["target"]

target                                                 1.000000
target_min                                             0.818803
target_max                                             0.630801
target_variance                                        0.184818
target_count                                           0.094135
precipitable_water_entire_atmosphere                  -0.031329
relative_humidity_2m_above_ground                     -0.084835
specific_humidity_2m_above_ground                     -0.035670
temperature_2m_above_ground                           -0.038818
u_component_of_wind_10m_above_ground                  -0.148202
v_component_of_wind_10m_above_ground                  -0.055100
L3_NO2_NO2_column_number_density                       0.306772
L3_NO2_NO2_slant_column_number_density                 0.315718
L3_NO2_absorbing_aerosol_index                         0.030386
L3_NO2_cloud_fraction                                 -0.123956
L3_NO2_sensor_altitude                  

In [5]:
train_X = train.drop(train.columns[3:8], axis = 1)
train_y = train["target"]

In [6]:
cols = train_X.columns[3:-7]
pipe1 = make_pipeline(SimpleImputer(), StandardScaler())
colTrans = make_column_transformer((pipe1, cols))

In [8]:
train_X = colTrans.fit_transform(train_X)
test_t = colTrans.transform(test)

In [8]:
params1 = {"n_estimators": np.arange(100, 500, 10), "max_depth": np.arange(1,10, 2), "subsample": np.arange(0.5,1.0,0.1), "max_features": np.arange(10,60)}
# params = {"n_estimators": np.arange(100, 500, 10), "max_depth": np.arange(1,10, 2), "max_features": np.arange(0.1, 1.0, 0.1)}

rs1 = RandomizedSearchCV(GradientBoostingRegressor(random_state=123), params1, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)

# rs = RandomizedSearchCV(RandomForestRegressor(random_state=123), params, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)

rs1.fit(train_X, train_y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...e=123, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220,
       230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350,
       360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480,
       490]), 'max_depth': array([1, 3, 5, 7, 9...37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])},
          pre_dispatch='2*n_jobs', random_state=12

In [18]:
rs1.best_params_

{'subsample': 0.7999999999999999,
 'n_estimators': 490,
 'max_features': 19,
 'max_depth': 9}

In [12]:
est = rs1.best_estimator_
pred2 = est.predict(test_t)
get_submission(pred2, "bestGrad1")

In [ ]:
params2 = {"n_estimators": np.arange(500,1000, 10), "max_depth": np.arange(6,20, 2), "subsample": np.arange(0.6,1.0,0.1), "max_features": np.arange(18,50)}
rs2 = RandomizedSearchCV(GradientBoostingRegressor(random_state=123), params2, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)
rs2.fit(train_X, train_y)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [13]:
rs2.best_params_

{'subsample': 0.7999999999999999,
 'n_estimators': 610,
 'max_features': 41,
 'max_depth': 8}

In [20]:
np.sqrt(-rs2.best_score_)

33.809336661144414

In [19]:
est_best = rs2.best_estimator_
prediction = est_best.predict(test_t)
get_submission(prediction, "bestGradientBoost")